In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x


In [2]:
from nltk.lm import NgramCounter
from nltk.util import ngrams

from google.colab import drive
#drive.mount('/content/gdrive')
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import json
import konlpy
from nltk.tokenize import word_tokenize
import re
from konlpy.tag import Okt
from konlpy.tag import Mecab, Komoran
import time
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from itertools import combinations
from itertools import product
import math
import pickle
"""
import sys
sys.setrecursionlimit(10**7)
"""

'\nimport sys\nsys.setrecursionlimit(10**7)\n'

In [3]:
#komoran=Komoran() mecab 만 사용할 거야. 이 칸은 텍스트 데이터 전처리에 관한 것.
mecab=Mecab()

pattern_1=r"\([^)]*\)"
pattern_2=r"\.{2}"
#json file 을 더블리스트 형태로
def preprocess_sent(lst_text):
    wholelist=[]
    buf=str()
    subkor_1=re.sub(pattern=pattern_1, repl="", string=lst_text)
    subkor_2=re.sub(pattern=pattern_2, repl="", string=subkor_1)
    clear_list=re.compile("[가-힣]+|\.{1}").findall(subkor_2)
    for word in clear_list:
        if word==".":
            buf+=(word+" ")
            wholelist.append(buf)
            buf=str()
        
        else:
            buf+=(word+" ")
    return wholelist

#더블리스트 형태를 형태소 분석된 채의 더블리스트 형태로 
def sent_to_morph(lst_sent):
    double_list=[]
    for sentence in lst_sent:
        #listed=komoran.morphs(sentence)
        listed=mecab.morphs(sentence)
        double_list.append(listed)
    return double_list

In [4]:
#윗 셀에 대한 예시. 파일 하나 가지고 놀기. 
with open("/content/gdrive/MyDrive/origin/BWSC217000049025.json", "r") as f:
    json_data=json.load(f)
#print(json.dumps(json_data))
kor_text=json_data["SJML"]["text"][1]["content"]
listed=preprocess_sent(kor_text)
double=sent_to_morph(listed)
print(double)


[['고객', '의', '경우', '올레', '를', '단기간', '이용', '할', '수', '있', '는', '올레', '플래시', '상품', '을', '포인트', '로', '결제', '할', '수', '있', '게', '된다', '.'], ['.'], ['.'], ['는', '자사', '의', '프리미엄', '멤버십', '서비스', '인', '올레', '클럽', '회원', '을', '대상', '으로', '올레', '플래시', '상품', '을', '포인트', '인', '별로', '차감', '하', '는', '서비스', '를', '제공', '한다고', '일', '밝혔', '다', '.'], ['.'], ['.'], ['올레', '서비스', '는', '필요', '할', '때', '마다', '시간', '또는', '하루', '단위', '로', '구매', '하', '는', '서비스', '다', '.'], ['한', '시간', '은', '원', '하루', '는', '원', '이', '다', '.'], ['올레', '접속', '홈페이지', '에서', '결제', '후', '바로', '이용', '할', '수', '있', '다', '.'], ['또', '가로수길', '홍대', '삼청동', '등', '가맹점', '카페', '의', '매주', '수요일', '할인', '혜택', '을', '오', '는', '월', '말', '까지', '연장', '한다', '.'], ['하루', '에', '회', '이용', '한', '금액', '만큼', '별', '이', '차감', '된다', '.'], ['이용', '할', '수', '있', '는', '카페', '의', '위치', '등', '은', '올레', '클럽', '홈페이지', '에서', '확인', '할', '수', '있', '다', '.'], ['.'], ['.'], ['이', '외', '에', '도', '지난', '월', '부터', '데이터', '가', '부족', '한', '고객', '들', '이', '추가', '데이터', '를', '이용'

In [5]:
#학습된 모델을 바탕으로 단순히 숫자 세는 모델. 여기서는 ngramcounters 기준. 


def bigram_probs(model, pre_word, post_word):
    word_dict=dict(model[[pre_word]])
    wholecount=model[pre_word]
    if wholecount!=0:

      try:
          post_count=word_dict[post_word]
      except:
          post_count=0
     

      emer_prob=post_count/wholecount

    else:
      emer_prob=0
    return emer_prob

def trigram_probs(model, pre_word, post_word):
    tulist=[]
    word_dict=dict(model[[pre_word]])
    
    for key in list(word_dict.keys()):
        if key !=pre_word:
            first_prob=bigram_probs(model, pre_word, key)
            second_prob=bigram_probs(model, key, post_word)
            whole_prob=first_prob*second_prob
            if whole_prob!=0:
                #normalization
                whole_prob=whole_prob*2
                tulist.append([pre_word, key, post_word, whole_prob])
        else:
            continue
    return tulist

In [6]:
#위에서 숫자 센 걸 바탕으로 
def canconnect(model, pre_word, post_word):
    possiblelist=[]
    biprob=bigram_probs(model, pre_word, post_word)
    if biprob!=0:
        possiblelist.append([pre_word, post_word, biprob])
    
    tulist=trigram_probs(model, pre_word, post_word)
    for li in tulist:
        possiblelist.append(li)
    return possiblelist
#  가능한 케이스들과 이에 대한 확률 제시. 인덱스도 만들어야 해(가능한 경우의 수 개수)
def bigconnect(model, list_of_words):
    dummylist=[]
    index_list=[]
    for i in range(len(list_of_words)-1):
        connection=canconnect(model, list_of_words[i], list_of_words[i+1])
        if len(connection)!=0:
            n=len(connection)
            index_list.append(n)
            for possible in connection:
                dummylist.append(possible)
    
        else:
            print("connection failed")
            break
    return dummylist, index_list


In [7]:
#위에서 주어진 dummylist 와 index_list를 바탕으로 깔끔한 출력을 위한 함수들.
def process(model, list_of_words):
    doublelists, index=bigconnect(model, list_of_words)
    for indexes in range(len(doublelists)):
        doublelists[indexes]=tuple(doublelists[indexes])
    return doublelists, index

def separate_connection(model,list_of_words):
    newconlist=[]
    pro_doublelists, pro_index=process(model, list_of_words)
    end, start=0, 0
    for i in pro_index:
        end+=i
        newconlist.append(pro_doublelists[start:end])

        start+=i
    return newconlist
  

#확률값이 너무 작아서 별로니 이를 예쁘게. 이 점수의 최댓값은 10000. 최저는 0. 
def pretty_prob(prob):
  prob=10000*prob
  prob=round(prob, 4)
  return prob  

def make_it_one(model, list_of_words):
    wholeconcat=[]
    wholeprob=1.0
    newlist=separate_connection(model, list_of_words)
    #separate connection을 거치면 
    maybe_one=list(product(*newlist))
    for i in maybe_one:
        for index in range(len(i)):
            wholeconcat.append(i[index][:-2])
            wholeprob*=i[index][-1]
        wholeconcat.append(tuple(list_of_words[-1]))
        wholeconcat.append(pretty_prob(wholeprob))
        wholeprob=1.0
    return wholeconcat



In [13]:
n_model=NgramCounter()
def ngrams_mine(double_list, model):
  text_unigrams = [ngrams(sent, 1) for sent in double_list]
  text_bigrams  = [ngrams(sent, 2) for sent in double_list]
  model.update(text_unigrams+text_bigrams)
  return model


#지금부터는 예시
text=[["a", "b", "c", "d"], ["d", "w", "c", "a"], ["c", "b", "d", "a"]]
ngrams_mine(text, n_model)
text2=[["a", "b", "c", "D"], ["d", "K", "c", "a"], ["c", "b", "C", "a"]]
ngrams_mine(text, n_model)

In [9]:
#학습 phase. 일단 그냥 전체 데이터에 대한 리스트질을 해보자
#파일 받아서 전체과정
def file_to_string(filenick):
    dummylist=[]

    with open(filenick) as f:
        json_data=json.load(f)
    for i in range(0, 1000):
        try:
        
            kor_text=json_data["SJML"]["text"][i]["content"]
            dummylist.append(kor_text)
        except:
            break
    return dummylist

def pre_pipeline(stringed_text):
    listed=preprocess_sent(stringed_text)
    double=sent_to_morph(listed)
    
    return double

def whole_pipeline(filenick, given_model):
    dummies=file_to_string(filenick)
    
    for dummy in dummies:
        
        double_list=pre_pipeline(dummy)
        ngrams_mine(double_list, given_model)

In [ ]:
#실제 학습 phase
"""
d_path="미정"
filelist=os.listdir(d_path)
for filename in filelist:
    uniquefile=os.path.join(d_path, filename )
"""

In [16]:
#생성된 객체 저장하는 시스템 만들기 (pickle 활용)
"""
with open("/content/gdrive/MyDrive/origin/testjames.p", "wb") as file:
  pickle.dump(n_model, file)
"""

In [18]:
#생성된 객체 불러오는 시스템 만들기 (pickle 활용)
"""
with open("/content/gdrive/MyDrive/origin/testjames.p", "rb") as file:
  maybe=pickle.load(file)

maybe.N()
"""

'\nwith open("/content/gdrive/MyDrive/origin/testjames.p", "rb") as file:\n  maybe=pickle.load(file)\n\nmaybe.N()\n'

1